### Init

Sample encoded database for better performance

In [ ]:
dataSize = 3000
indexes = sample(1:size(encodedDB, 2), dataSize, replace=false)
data = getCol(encodedDB, indexes)
outDim = 2

Use the whole database

In [ ]:
indexes = 1:size(encodedDB, 2)
data = encodedDB
outdim = 2

### PCA

In [ ]:
DimRedMethod = "PCA"
model = fit(PCA, data; maxoutdim=outDim)
pred = MultivariateStats.transform(model, data)

### T-sne

In [ ]:
DimRedMethod = "T-sne"
reduce_dims = 0
max_iter = 3000
perplexity = 100.0
pred = permutedims(tsne(distances[indexes, indexes], distance=true, outDim, reduce_dims, max_iter, perplexity))

# Clustering

The closer the silhouette value is to 1.0 the better the clustering is

### Init

In [ ]:
cluster_count=8

### Based on first first in preference

In [ ]:
ClustMethod = "Party"
labels = database[1, indexes]
clusters = clusterize(labels, candidates, parties)
mean(silhouettes(labels, distances[indexes, indexes]))

### K-means

In [ ]:
ClustMethod = "K-means"
KmeansRes = kmeans(data, cluster_count; maxiter=200)
labels = KmeansRes.assignments
clusters = clusterize(labels)
mean(silhouettes(labels, distances[indexes, indexes]))

### Gaussian mixtures

In [ ]:
ClustMethod = "GM"
data_T = permutedims(data)
gm = GaussianMixture(n_components=cluster_count).fit(data_T)
labels = gm.predict(data_T) .+ 1
clusters = clusterize(labels)
mean(silhouettes(labels, distances[indexes, indexes]))

## Clustering Validation

In [ ]:
mean(silhouettes(labels, distances[indexes, indexes]))

## Clustering visualization

Save template clusters for later to match cluster colours based on it

In [ ]:
template = clusters

In [ ]:
unify_labels!(template, clusters)

In [ ]:
visualize(pred, clusters, DimRedMethod, ClustMethod, output=true)

### Creating graph out of clustered database

In [ ]:
@time G = createClusteredMetaGraph(g, clusters, labels)

## Drawing clustered graph

In [ ]:
@time drawClusteredMetaGraph(G)

# Main ________________________________

In [1]:
using Revise

In [2]:
using OpinionDiffusion

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-18298259272998564304\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-946369705831781315\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-18298259272998564304\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [ ]:
@sk_import mixture : GaussianMixture
Base.show(io::IO, f::Float64) = @printf(io, "%1.4f", f)

In [3]:
OpinionDiffusion.Plots.PlotlyBackend()

Plots.PlotlyBackend()

Parse input data

In [4]:
input_filename = "ED-00001-00000002.toc"
@time parties, candidates, election = parse_data2(input_filename)

  0.205552 seconds (568.52 k allocations: 55.975 MiB, 20.93% gc time, 7.01% compilation time)


(["G.P.", "Lab", "F.F.", "S.P.", "S.F.", "P.D.", "Csp", "F.G."], OpinionDiffusion.Candidate[OpinionDiffusion.Candidate("Robert Bonnie", 1), OpinionDiffusion.Candidate("Joan Burton", 2), OpinionDiffusion.Candidate("Deirdre Doherty Ryan", 3), OpinionDiffusion.Candidate("Joe Higgins", 4), OpinionDiffusion.Candidate("Brian Lenihan", 3), OpinionDiffusion.Candidate("Mary Lou Mc Donald", 5), OpinionDiffusion.Candidate("Tom Morrissey", 6), OpinionDiffusion.Candidate("John Thomas Smyth C.C.", 7), OpinionDiffusion.Candidate("Sheila Terry", 8)], [[[5], [3], [7], [1, 2, 4, 6, 8, 9]], [[5], [3], [7], [1, 2, 4, 6, 8, 9]], [[5], [3], [7], [1, 2, 4, 6, 8, 9]], [[5], [3], [7], [1, 2, 4, 6, 8, 9]], [[5], [3], [7], [1, 2, 4, 6, 8, 9]], [[5], [3], [7], [1, 2, 4, 6, 8, 9]], [[5], [3], [7], [1, 2, 4, 6, 8, 9]], [[5], [3], [7], [1, 2, 4, 6, 8, 9]], [[5], [3], [7], [1, 2, 4, 6, 8, 9]], [[5], [3], [7], [1, 2, 4, 6, 8, 9]]  …  [[7], [5], [4], [3], [2], [1], [9], [8], [6]], [[4], [2], [7], [9], [3], [5], [1, 6, 

In [5]:
model_config = Dict(
    "weight_func" => Dict(
        "type" => "exp",
        "base" => 1/2
    ),
    "dist_metric" => "L1",
    "edge_init_func" => Dict(
        "type" => "exp",
        "base" => 1/2,
        "offset" => -6.28
    )
)

Dict{String, Any} with 3 entries:
  "weight_func"    => Dict{String, Any}("base"=>0.5, "type"=>"exp")
  "dist_metric"    => "L1"
  "edge_init_func" => Dict{String, Any}("base"=>0.5, "offset"=>-6.28, "type"=>"…

In [ ]:
model = Spearman_model(election, length(candidates), model_config)

In [ ]:
model.log_dir

In [6]:
model = OpinionDiffusion.load("logs/2021-06-24_15-40-59/model.jld2", "model")

Spearman_model(OpinionDiffusion.Spearman_voter[OpinionDiffusion.Spearman_voter(1, [0.05054432348367029, 0.05054432348367029, 0.19906687402799378, 0.05054432348367029, 0.39813374805598756, 0.05054432348367029, 0.09953343701399689, 0.05054432348367029, 0.05054432348367029], 0.5236957031079401, 0.4138570715231003), OpinionDiffusion.Spearman_voter(2, [0.05054432348367029, 0.05054432348367029, 0.19906687402799378, 0.05054432348367029, 0.39813374805598756, 0.05054432348367029, 0.09953343701399689, 0.05054432348367029, 0.05054432348367029], 0.2774860712520548, 0.437116248727071), OpinionDiffusion.Spearman_voter(3, [0.05054432348367029, 0.05054432348367029, 0.19906687402799378, 0.05054432348367029, 0.39813374805598756, 0.05054432348367029, 0.09953343701399689, 0.05054432348367029, 0.05054432348367029], 0.4768312678910844, 0.3641331927200218), OpinionDiffusion.Spearman_voter(4, [0.05054432348367029, 0.05054432348367029, 0.19906687402799378, 0.05054432348367029, 0.39813374805598756, 0.0505443234

In [7]:
exp_config = Dict(
    "sample_size" => 3000,
    "voter_visualization_config" => Dict(
        "used" => true,
        "reduce_dim_config" => Dict(
            "used" => true,
            "method" => "PCA",
            "PCA" => Dict(
                "out_dim" => 2
            ),
            "tsne" => Dict(
                "out_dim" => 2,
                "reduce_dims" => 0,
                "max_iter" => 3000,
                "perplexity" => 100.0
            )
        ),
        "clustering_config" => Dict(
            "used" => true,
            "method" => "Party",
            "K-means" => Dict(
                "cluster_count" => 8
            ),
            "GM" => Dict(
                "cluster_count" => 8
            )
        )
    )
)

Dict{String, Any} with 2 entries:
  "voter_visualization_config" => Dict{String, Any}("used"=>true, "reduce_dim_c…
  "sample_size"                => 3000

In [15]:
experiment = Experiment(model, candidates, parties, OpinionDiffusion.Plots.PlotlyBackend, exp_config)

LoadError: UndefVarError: jldsave not defined

In [9]:
diffusion_config = Dict(
        "diffusions" => 5,
        "checkpoint" => 1,
        "voter_diff_config" => Dict(
            "evolve_vertices" => 100000,
            "method" => "averageAll"
        ),
        "edge_diff_config" => Dict(
            "evolve_edges" => 100000,
            "dist_metric" => "L1",
            "edge_diff_func" => Dict(
                "type" => "exp",
                "base" => 1/2
            )
        )
    )

Dict{String, Any} with 4 entries:
  "edge_diff_config"  => Dict{String, Any}("dist_metric"=>"L1", "evolve_edges"=…
  "voter_diff_config" => Dict{String, Any}("method"=>"averageAll", "evolve_vert…
  "diffusions"        => 5
  "checkpoint"        => 1

In [10]:
diffusion_metrics = run_experiment!(experiment, candidates, parties, diffusion_config)

LoadError: UndefVarError: experiment not defined

In [ ]:
OpinionDiffusion.visualize_metrics(experiment, candidates, parties)

In [ ]:
using Reexport
using Interact

In [ ]:

px=widget(0:0.01:.3, label="px")
hbox(px)

In [ ]:
OpinionDiffusion.plot(experiment.visualizations.voter_visualizations[step], experiment.visualizations.degree_distributions[step], layout = (2, 1), size = (980,1200))

In [ ]:
dist_metric = Euclidean()
@time distances = pairwise(distMetric, [voter.opinion for voter in experiment.sampled_voters], dims=2)

In [ ]:
labels = [voter.label for voter in experiment.sampled_voters]
mean(silhouettes(labels, distances))

In [ ]:
logdir = "logs/" * Dates.format(now(), "yyyy-mm-dd_HH-MM-SS")
expCounter = 1
mkpath(logdir)

In [ ]:
weightFunc = parseFunction(initConfig["weightFunc"])
weights = map(weightFunc, 1.0:length(candidates))
weights = translateRange(minimum(weights), maximum(weights), 0.0, 1.0, weights)

In [ ]:
distMetric = parseMetric(initConfig["distMetric"])
@time distances = pairwise(distMetric, opinions, dims=2)